- 단계구분도: 경기도 내 인구소멸 비율

In [1]:
import json, folium
import numpy as np
import pandas as pd

- 경기도 시군구별 인구소멸비율 구하기_'99.인구소멸비율' 파일 참조

In [2]:
pop = pd.read_csv('../02.pandas/data/주민등록인구집계현황.csv',encoding='euc=kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수','60~69세','70~79세','80~89세','90~99세','100세 이상']]
pop['인구소멸비율'] = pop.iloc[:, 2:7].sum(axis=1) / pop.iloc[:, -2:].sum(axis=1).round(3)
pop=pop[['행정구역구분명','행정구역명','총 인구수','인구소멸비율']]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
0,도,경기도,13589432,277.819752
1,시군,경기도 가평군,62197,149.497427
2,읍면동,경기도 가평군 가평읍,19555,155.888235
3,읍면동,경기도 가평군 북면,3815,106.763636
4,읍면동,경기도 가평군 상면,5669,148.438596


In [3]:
pop= pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,149.497427
8,시군,경기도 고양시,1075202,234.945067
9,구,경기도 고양시 덕양구,487874,253.995376
31,구,경기도 고양시 일산동구,296590,222.495700
44,구,경기도 고양시 일산서구,290738,219.461063


In [4]:
# 수원시, 용인시 등 구가 있는 시는 제거
gu_city = ['수원시','고양시','용인시','성남시','안산시','안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:    # split의 -1자리 값이 gu_city중에 있다면
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,149.497427
9,구,경기도 고양시 덕양구,487874,253.995376
31,구,경기도 고양시 일산동구,296590,222.495700
44,구,경기도 고양시 일산서구,290738,219.461063
56,시군,경기도 과천시,77775,222.872941


- geo data 읽기

In [5]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

- 인구데이터와 geo 데이터간 인덱스 일치시키기

In [6]:
# 참)경기도_geo_simple.json 파일에서 시 시구 군으로 되어있음 확인
pop['시군구']= pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율,시군구
1,시군,경기도 가평군,62197,149.497427,가평군
9,구,경기도 고양시 덕양구,487874,253.995376,고양시덕양구
31,구,경기도 고양시 일산동구,296590,222.495700,고양시일산동구
44,구,경기도 고양시 일산서구,290738,219.461063,고양시일산서구
56,시군,경기도 과천시,77775,222.872941,과천시


In [7]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,149.497427
고양시덕양구,구,경기도 고양시 덕양구,487874,253.995376
고양시일산동구,구,경기도 고양시 일산동구,296590,222.495700
고양시일산서구,구,경기도 고양시 일산서구,290738,219.461063
과천시,시군,경기도 과천시,77775,222.872941


- 단계 구분도

In [8]:
map = folium.Map([37.4,127.2],zoom_start=9,tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,                          # 단계구분도로 보여줄 데이터
    columns=[pop.index, pop.인구소멸비율],            # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',                    # Colormap
    key_on='properties.name'   # 경기도_geo_simple파일에서
).add_to(map)
map